# HJM model with stochastic volatility (CIR)

This notebook implements a single-factor HJM forward-rate model with CEV scaling and a single mean-reverting stochastic volatility factor, calibrated to caplet and swaption markets.  



### Model

**Forward-rate SDE (risk-neutral measure $(\mathbb{Q})$):**
$$
{\;df(t,T) \;=\; \mu(t,T)\,dt \;+\; \sigma(t,T)\,dW_t\;}
$$

**_where:_**

Instantaneous forward volatility (CEV × amplitude):
$$
{\sigma(t,T) = v_t\,\phi(t,T)\,\bigl[f(t,T)+\delta\bigr]^{\beta}}
$$
where: 
- $f(t,T)$ = instantaneous forward rate maturing at $T$ observed at time $t$.  
- $v_t$ = scalar stochastic volatility (amplitude).  
- $\phi(t,T)$ = deterministic shape (term-structure), e.g. $\phi(t,T) = a^{-b(T-t)}$.  
- $\beta$ = CEV exponent.  
- $\delta \geq 0$ = small shift (default $1\times 10^{-4}$) to handle near-zero/negative rates.  

where:

**Stochastic volatility SDE (CIR / Heston-like):**
$$
{\;dv_t \;=\; \kappa(\theta - v_t)\,dt \;+\; \xi\sqrt{v_t}\,dZ_t\;}
$$
where:
- $\kappa$ = speed of mean reversion, $\theta$ = long-run level, $\xi$ = vol-of-vol.  
- Use Andersen QE exact/approx scheme where possible; fallback to full-truncation Euler for robustness.

**Correlation between the driving noises:**
The two Brownian motions $W_t$ (for forwards) and $Z_t$ (for volatility) satisfy
$$
{\;d\langle W,Z\rangle_t \;=\; \rho\,dt\;}
$$
with $\rho \in [-1,1]$ (typical bounds used in calibration code: $[-0.99,0.99]$).

**HJM no-arbitrage drift:**
$$
{\;\mu(t,T) \;=\; \sigma(t,T)\,\int_{t}^{T}\sigma(t,s)\,ds\;.}
$$
- In practice the integral $\int_t^T \sigma(t,s)\,ds$ is approximated numerically on the model $T$-grid (trapezoid / Simpson).



Deterministic shape (example parameterisation):
A simple, calibratable choice used in the code:
$$
\phi(t,T) = a \, e^{-b (T - t)} 
$$
with $a > 0$, $b \ge 0$ exposed as calibration parameters.



### Process

**1. Configuration & environment**  
   - CONFIG dictionary: random number generator (RNG) seed, MC path counts, grid resolution, weights for swaption vs caplet, file paths.

**2. Synthetic market data generation** (`/data/*.csv`)  
   - Create deterministic OIS/RFR curve CSV (`curve.csv`) with times, zero rates and discount factors.  
   - Create synthetic caplet vol matrix (`caplet_quotes.csv`) and synthetic swaption surface (`swaption_quotes.csv`).  
   - Files are deterministic (seeded) so runs are reproducible.

**3. Bootstrapping / interpolation**  
   - Read `curve.csv`, build interpolant \(P(0,t)\) (log-linear interpolation of discount factors).  
   - Compute initial instantaneous forward curve $f(0,T) = -\partial_T \ln P(0,T)$ on a fixed $T$-grid.

**4. Model initialisation**  
   - Instantiate `HJMModel` with: T-grid, initial forwards $f(0,T)$, parameter struct ($a, b, \beta, \delta, \kappa, \theta, \xi, \rho, v_0$)
, and $\phi(t,T)$ function.

**5. Monte-Carlo simulation engine**  
   - Simulate forward-curve paths and the volatility factor $(v_t)$ using correlated normals (common random numbers, optional antithetic variates).  
   - Use full-truncation Euler for $v_t$.  
   - Compute HJM drift $\mu(t,T)$ via numerical quadrature on the $T$-grid at each step.  

**6. Instrument pricers (MC-based)**  
   - **Caplet pricer:** uses simulated $f(\text{expiry}, \text{expiry}+\tau)$ to compute payoff $\mathrm{DF}\,\tau\,\max(F-K,0)$.
   - **Swaption pricer:** approximate via forward swap rate and annuity (MC used to evolve state to expiry where needed).  
   - Provide Black implied-vol inversion utilities to compare model prices to quoted vols.

**7. Calibration — two stages**  
   - **Stage A (ATM term structure):** calibrate $(a,b,\beta)$ to match ATM caplet & ATM swaption term structures (fast MC or approximations).  
  Swaption vs caplet weights are configurable; default places higher weight on swaption surface for mid/long tenors.  

   - **Stage B (smiles):** fix Stage A parameters and calibrate $(\kappa,\theta,\xi,\rho,v_0)$ to capture smile/skew across strikes/deltas using Monte Carlo pricing.  
  Use global $\rightarrow$ local optimisation strategy in practice (the notebook uses local methods for speed).  


**8. Diagnostics & outputs**  
   - Save: `outputs/calibration_results.json`, `outputs/calibrated_parameters.json`, `outputs/model_vs_market_caplets.csv`, `outputs/model_vs_market_swaptions.csv`.  
   - Plots: ATM caplet fit, ATM swaption surface fit, selected smile comparisons (PNG files under `outputs/plots/`).

In [5]:
# ----------------------------
# Dependencies & CONFIG
# ----------------------------
import os
import json
import math
import shutil
from dataclasses import dataclass
from typing import Callable, Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate, optimize, stats
from scipy.optimize import brentq
from scipy.integrate import cumulative_trapezoid

# ---------- CONFIG ----------
CONFIG = {
    "currency": "USD",                # SOFR -> USD. Use "GBP"->SONIA, "EUR"->ESTR
    "seed": 42,
    "n_paths_stageA": 2,              # demo; 2 instead of 2000 for speed/illustration on personal laptop
    "n_paths_stageB": 3,              # increase for more accurate Stage B; 3 instead of 12000 number for speed/illustration on personal laptop
    "n_steps_per_year": 4,            # 4 instead of 12; unrealistically low to save processing speed on personal laptop
    "swaption_weight": 0.7,
    "cap_weight": 0.3,
    "fast_mode": True,
    "phi_init": {"a": 0.02, "b": 0.5},
    "beta_init": 0.0,
    "vol_shift": 1e-4,
    "qe_scheme": False,               # not used; full-truncation Euler used
    "output_dir": "outputs",
    "data_dir": "data",
}
np.random.seed(CONFIG["seed"])

In [7]:
# --------------------
# Utilities & IO Setup
# --------------------
def ensure_dirs():
    for d in [CONFIG["data_dir"], CONFIG["output_dir"], os.path.join(CONFIG["output_dir"], "plots")]:
        os.makedirs(d, exist_ok=True)
ensure_dirs()

def tenor_to_years(tenor: str) -> float:
    """Convert tenor like '1D','1W','1M','3M','6M','1Y','5Y' to years (approx)."""
    tenor = tenor.strip().upper()
    if tenor.endswith('D'):
        return int(tenor[:-1]) / 365.0
    if tenor.endswith('W'):
        return int(tenor[:-1]) * 7 / 365.0
    if tenor.endswith('M'):
        return int(tenor[:-1]) / 12.0
    if tenor.endswith('Y'):
        return int(tenor[:-1])
    raise ValueError("Unknown tenor format: " + tenor)

def save_csv(df: pd.DataFrame, path: str):
    df.to_csv(path, index=False)
    print(f"Saved: {path}")

In [9]:
# -----------------------
# Dummy market data files
# -----------------------
def generate_dummy_curve(path: str, currency: str = "USD", base_rate: float = 0.02):
    """Generate a deterministic OIS/RFR-like curve and save to CSV."""
    tenors = ["1M","3M","6M","1Y","2Y","3Y","5Y","7Y","10Y","15Y","20Y","30Y"]
    times = np.array([tenor_to_years(t) for t in tenors])
    zero_rates = base_rate + 0.002 * np.log1p(times * 10)
    discount_factors = np.exp(-zero_rates * times)
    df = pd.DataFrame({
        "tenor": tenors,
        "time": times,
        "zero_rate": zero_rates,
        "discount_factor": discount_factors,
        "currency": currency,
        "date": pd.Timestamp.today().strftime("%Y-%m-%d"),
        "rate_type": "continuous"
    })
    save_csv(df, path)
    return df

def generate_dummy_caplet_vols(path: str, currency: str = "USD"):
    """Generate synthetic caplet vol quotes: expiries x strikes."""
    expiries = ["6M","1Y","2Y","3Y","5Y","7Y"]
    strikes = [-0.005, 0.0, 0.01, 0.02, 0.03]
    rows = []
    atm_base = {"6M":0.008, "1Y":0.01, "2Y":0.015, "3Y":0.017, "5Y":0.02, "7Y":0.023}
    for e in expiries:
        for k in strikes:
            atm = atm_base[e]
            vol = atm * (1.0 + 0.5 * (0.02 - k)) + 0.001 * abs(k)
            rows.append({"expiry": e, "strike": round(k,4), "vol": max(vol, 0.0005), "vol_type": "normal", "currency": currency})
    df = pd.DataFrame(rows)
    save_csv(df, path)
    return df

def generate_dummy_swaption_vols(path: str, currency: str = "USD"):
    """Generate synthetic swaption vol surface: expiry x tenor x moneyness"""
    expiries = ["6M","1Y","2Y","3Y","5Y","10Y"]
    tenors = ["1Y","2Y","5Y","10Y"]
    moneyness = ["-25D","ATM","25D"]
    rows = []
    atm_table = {
        ("6M","1Y"):0.007, ("6M","2Y"):0.008, ("6M","5Y"):0.01, ("6M","10Y"):0.012,
        ("1Y","1Y"):0.009, ("1Y","2Y"):0.011, ("1Y","5Y"):0.013, ("1Y","10Y"):0.015,
        ("2Y","1Y"):0.011, ("2Y","2Y"):0.013, ("2Y","5Y"):0.015, ("2Y","10Y"):0.017,
        ("3Y","1Y"):0.012, ("3Y","2Y"):0.014, ("3Y","5Y"):0.016, ("3Y","10Y"):0.018,
        ("5Y","1Y"):0.013, ("5Y","2Y"):0.015, ("5Y","5Y"):0.018, ("5Y","10Y"):0.02,
        ("10Y","1Y"):0.016, ("10Y","2Y"):0.018, ("10Y","5Y"):0.02, ("10Y","10Y"):0.022
    }
    for e in expiries:
        for t in tenors:
            atm = atm_table.get((e,t), 0.014)
            for m in moneyness:
                if m == "ATM":
                    vol = atm
                elif m == "-25D":
                    vol = atm * 1.12
                else:
                    vol = atm * 0.92
                rows.append({"expiry": e, "tenor": t, "moneyness": m, "vol": max(vol, 1e-4), "vol_type": "black", "currency": currency})
    df = pd.DataFrame(rows)
    save_csv(df, path)
    return df

# create files (if absent)
curve_csv = os.path.join(CONFIG["data_dir"], "curve.csv")
caplet_csv = os.path.join(CONFIG["data_dir"], "caplet_quotes.csv")
swaption_csv = os.path.join(CONFIG["data_dir"], "swaption_quotes.csv")

if not os.path.exists(curve_csv):
    generate_dummy_curve(curve_csv, currency=CONFIG["currency"])
if not os.path.exists(caplet_csv):
    generate_dummy_caplet_vols(caplet_csv, currency=CONFIG["currency"])
if not os.path.exists(swaption_csv):
    generate_dummy_swaption_vols(swaption_csv, currency=CONFIG["currency"])

Saved: data\curve.csv
Saved: data\caplet_quotes.csv
Saved: data\swaption_quotes.csv


In [25]:
# -------------------------------
# Bootstrapping helpers (simple)
# -------------------------------
def read_curve(path: str):
    df = pd.read_csv(path)
    df = df.sort_values("time").reset_index(drop=True)
    return df

def interp_discount(df_curve: pd.DataFrame) -> Callable[[float], float]:
    """Return a function P(0,t) by linear interpolation in log DF space."""
    times = df_curve["time"].values
    dfs = df_curve["discount_factor"].values
    ln_dfs = np.log(np.maximum(dfs, 1e-16))
    interp_fn = interpolate.interp1d(times, ln_dfs, kind="linear", fill_value="extrapolate")
    return lambda t: float(np.exp(interp_fn(t)))

def forward_from_discount(P: Callable[[float], float], times: np.ndarray) -> np.ndarray:
    """Finite difference estimate of instantaneous forward rates at supplied times."""
    f = []
    for t in times:
        h = max(1e-5, min(0.01, t*1e-2))
        if t - h < 0:
            val = -(math.log(P(t+h)) - math.log(P(t))) / h
        else:
            val = -(math.log(P(t+h)) - math.log(P(t-h))) / (2*h)
        f.append(float(val))
    return np.array(f)

curve_df = read_curve(curve_csv)
P0 = interp_discount(curve_df)
curve_times = curve_df["time"].values
f0_times = np.linspace(0.01, max(curve_times)*1.0, num=60)   # model T-grid
f0_vals = forward_from_discount(P0, f0_times)

In [29]:
# ----------------------------------------
# HJM Model: discretised forward-curve MC
# ----------------------------------------
@dataclass
class HJMParams:
    a: float
    b: float
    beta: float
    delta: float
    kappa: float
    theta: float
    xi: float
    rho: float
    v0: float

class HJMModel:
    """Single-factor HJM with CEV scaling and one stochastic volatility factor v_t."""
    def __init__(self, times: np.ndarray, f0: np.ndarray, params: HJMParams,
                 phi_fn: Optional[Callable[[float, float], float]] = None,
                 n_steps_per_year: int = 4, seed: int = 42):    # 4 instead of 12 n_steps_per_year to save processor speed on personal laptop
        self.times = np.array(times)
        self.f0 = np.array(f0)
        self.params = params
        self.n_steps_per_year = n_steps_per_year
        self.dt = 1.0 / n_steps_per_year
        self.seed = seed
        np.random.seed(seed)
        self.Tgrid = self.times.copy()
        if phi_fn is None:
            self.phi_fn = lambda t, T: params.a * np.exp(-params.b * (T - t)) if T >= t else 0.0
        else:
            self.phi_fn = phi_fn

    def sigma(self, t_idx: Optional[int], t: float, f_vec: np.ndarray, v_t: float) -> np.ndarray:
        sig = np.zeros_like(f_vec, dtype=float)
        for i, T in enumerate(self.Tgrid):
            if T >= t:
                phi = self.phi_fn(t, T)
                base = max(f_vec[i] + self.params.delta, 0.0)
                sig[i] = v_t * phi * (base ** self.params.beta)
            else:
                sig[i] = 0.0
        return sig

    def hjm_drift(self, t: float, f_vec: np.ndarray, v_t: float) -> np.ndarray:
        """
        Compute μ(t,T) = σ(t,T) * ∫_t^T σ(t,s) ds using cumulative trapezoid on the model T-grid.
        Uses scipy.integrate.cumulative_trapezoid (fast, O(n)).
        """
        sig = self.sigma(None, t, f_vec, v_t)
        mu = np.zeros_like(sig)
        mask = self.Tgrid >= t
        if not np.any(mask):
            return mu
    
        Tsub = self.Tgrid[mask].copy()
        sig_sub = sig[mask].copy()
    
        # If earliest Tsub > t, estimate sigma(t,t) and prepend so integral starts at t
        prepended = False
        if Tsub[0] > t:
            interp_f = interpolate.interp1d(self.Tgrid, f_vec, kind="linear", fill_value="extrapolate")
            f_at_t = float(interp_f(t))
            base = max(f_at_t + self.params.delta, 0.0)
            sigma_at_t = v_t * self.phi_fn(t, t) * (base ** self.params.beta)
            Tsub = np.concatenate(([t], Tsub))
            sig_sub = np.concatenate(([sigma_at_t], sig_sub))
            prepended = True
    
        # cumulative_trapezoid from scipy.integrate returns integral[j] = ∫_{Tsub[0]}^{Tsub[j]} sig_sub(s) ds
        # import cumulative_trapezoid at the top: from scipy.integrate import cumulative_trapezoid
        integral = cumulative_trapezoid(sig_sub, Tsub, initial=0.0)
    
        # mu(t,T_j) = sigma(t,T_j) * integral_j
        mu_masked = sig_sub * integral
    
        # if we prepended a t point, the first element corresponds to T==t and integral==0, so drop it
        mu_vals = mu_masked[1:] if prepended else mu_masked
    
        mu[mask] = mu_vals
        return mu

    def simulate_paths(self, n_paths: int, Tsim: float, n_steps: Optional[int] = None, antithetic: bool = True):
        if n_steps is None:
            n_steps = max(1, int(np.ceil(Tsim * self.n_steps_per_year)))
        dt = Tsim / n_steps
        times = np.linspace(0.0, Tsim, n_steps+1)
        n_T = len(self.Tgrid)
        f_paths = np.zeros((n_paths, n_steps+1, n_T), dtype=float)
        v_paths = np.zeros((n_paths, n_steps+1), dtype=float)
        f_paths[:, 0, :] = np.tile(self.f0, (n_paths, 1))
        v_paths[:, 0] = self.params.v0

        rng = np.random.RandomState(self.seed)
        if antithetic:
            half = int(np.ceil(n_paths/2))
            Z1 = rng.normal(size=(half, n_steps))
            Z2 = rng.normal(size=(half, n_steps))
            Z1 = np.vstack([Z1, -Z1])[:n_paths, :]
            Z2 = np.vstack([Z2, -Z2])[:n_paths, :]
            eps_W = Z1
            eps_Z = Z2
        else:
            eps_W = rng.normal(size=(n_paths, n_steps))
            eps_Z = rng.normal(size=(n_paths, n_steps))

        rho = self.params.rho
        eps_Z = rho * eps_W + math.sqrt(max(0.0, 1.0 - rho**2)) * eps_Z

        for step in range(n_steps):
            t = times[step]
            sqrt_dt = math.sqrt(dt)
            for p in range(n_paths):
                f_cur = f_paths[p, step, :].copy()
                v_cur = max(v_paths[p, step], 0.0)
                sig_vec = self.sigma(step, t, f_cur, v_cur)
                mu_vec = self.hjm_drift(t, f_cur, v_cur)
                dW = sqrt_dt * eps_W[p, step]
                dZ = sqrt_dt * eps_Z[p, step]
                f_next = f_cur + mu_vec * dt + sig_vec * dW
                f_next = np.maximum(f_next, -0.99)
                kappa, theta, xi = self.params.kappa, self.params.theta, self.params.xi
                v_next = v_cur + kappa * (theta - v_cur) * dt + xi * math.sqrt(max(v_cur, 0.0)) * dZ
                v_next = max(v_next, 1e-12)
                f_paths[p, step+1, :] = f_next
                v_paths[p, step+1] = v_next
        return f_paths, v_paths, times

    # Pricing wrappers (bound to class)
    def price_caplet_mc(self, strike: float, expiry: float, accrual: float, payoff_df_fn: Callable[[float], float],
                        n_paths: int = 20, n_steps: Optional[int] = None) -> float:
        f_paths, v_paths, times = self.simulate_paths(n_paths=n_paths, Tsim=expiry, n_steps=n_steps, antithetic=True)
        target_T = expiry + accrual
        idx = np.argmin(np.abs(self.Tgrid - target_T))
        F_at_expiry = f_paths[:, -1, idx]
        DF_pay = payoff_df_fn(target_T)
        payoffs = np.maximum(F_at_expiry - strike, 0.0) * accrual * DF_pay
        price = np.mean(payoffs)
        return float(price)

    def price_swaption_mc(self, expiry: float, swap_tenor: float, fixed_rate: float, payment_freq: float,
                          payoff_df_fn: Callable[[float], float], n_paths: int = 20, n_steps: Optional[int] = None) -> float:
        f_paths, v_paths, times = self.simulate_paths(n_paths=n_paths, Tsim=expiry, n_steps=n_steps, antithetic=True)
        num_payments = int(round(swap_tenor * payment_freq))
        pay_times = np.array([expiry + (k+1)/payment_freq for k in range(num_payments)])
        df_payments = np.array([payoff_df_fn(pt) for pt in pay_times])
        accrual = 1.0 / payment_freq
        annuity = np.sum(df_payments) * accrual
        P_start = payoff_df_fn(expiry)
        P_end = payoff_df_fn(expiry + swap_tenor)
        S = (P_start - P_end) / annuity if annuity > 0 else 0.0
        price = P_start * max(S - fixed_rate, 0.0) * annuity
        return float(price)

In [31]:
# ------------------------
# Black & Bachelier helpers
# (Robust implied vol inversion)
# ------------------------
def black_caplet_price(F: float, K: float, sigma: float, tau: float, DF: float) -> float:
    if sigma <= 0 or tau <= 0:
        return DF * tau * max(F - K, 0.0)
    std = sigma * math.sqrt(tau)
    if std <= 0:
        return DF * tau * max(F - K, 0.0)
    if K <= 0 or F <= 0:
        return DF * tau * max(F - K, 0.0)
    d1 = (math.log(F / K) + 0.5 * std ** 2) / std
    d2 = d1 - std
    price = DF * tau * (F * stats.norm.cdf(d1) - K * stats.norm.cdf(d2))
    return float(price)

def bachelier_caplet_price(F: float, K: float, sigmaN: float, tau: float, DF: float) -> float:
    sigmaN_sqrt = sigmaN * math.sqrt(tau)
    if sigmaN_sqrt <= 0:
        return DF * tau * max(F - K, 0.0)
    x = (F - K) / sigmaN_sqrt
    price = DF * tau * ((F - K) * stats.norm.cdf(x) + sigmaN_sqrt * stats.norm.pdf(x))
    return float(price)

def implied_bachelier_vol_from_price_caplet(target_price, F, K, tau, DF, vol_min=1e-8, vol_max=5.0):
    price_at_0 = DF * tau * max(F - K, 0.0)
    price_at_inf = DF * tau * F
    if target_price <= price_at_0 + 1e-14:
        return 0.0
    if target_price >= price_at_inf - 1e-14:
        return vol_max
    def f(volN):
        return bachelier_caplet_price(F, K, volN, tau, DF) - target_price
    try:
        volN = brentq(f, vol_min, vol_max, maxiter=60, xtol=1e-12)
        return float(volN)
    except Exception:
        return float(np.nan)

def implied_black_vol_from_price_caplet(target_price, F, K, tau, DF, vol_min=1e-8, vol_max=5.0, tol=1e-12):
    price_at_0 = DF * tau * max(F - K, 0.0)
    price_at_inf = DF * tau * F
    if target_price <= price_at_0 + 1e-14:
        return 0.0
    if target_price >= price_at_inf - 1e-14:
        return vol_max
    if K <= 0.0 or F <= 0.0:
        return implied_bachelier_vol_from_price_caplet(target_price, F, K, tau, DF, vol_min, vol_max)
    def price_minus_target(sigma):
        return black_caplet_price(F, K, sigma, tau, DF) - target_price
    try:
        f_low = price_minus_target(vol_min)
        f_high = price_minus_target(vol_max)
        if f_low * f_high > 0:
            vol_max2 = 50.0
            f_high2 = price_minus_target(vol_max2)
            if f_low * f_high2 > 0:
                return implied_bachelier_vol_from_price_caplet(target_price, F, K, tau, DF, vol_min, vol_max)
            vol_upper = vol_max2
        else:
            vol_upper = vol_max
        vol_root = brentq(price_minus_target, vol_min, vol_upper, xtol=tol, maxiter=100)
        return float(vol_root)
    except Exception:
        return implied_bachelier_vol_from_price_caplet(target_price, F, K, tau, DF, vol_min, vol_max)

def black_swaption_price(S0: float, K: float, sigma: float, T: float, annuity: float) -> float:
    if sigma <= 0 or T <= 0:
        return annuity * max(S0 - K, 0.0)
    std = sigma * math.sqrt(T)
    if std <= 0:
        return annuity * max(S0 - K, 0.0)
    d1 = (math.log(S0 / K) + 0.5 * std ** 2) / std
    d2 = d1 - std
    price = annuity * (S0 * stats.norm.cdf(d1) - K * stats.norm.cdf(d2))
    return price

def implied_black_vol_from_price_swaption(S0: float, K: float, T: float, annuity: float, market_price: float):
    price_at_0 = annuity * max(S0 - K, 0.0)
    price_at_inf = annuity * S0
    if market_price <= price_at_0 + 1e-14:
        return 0.0
    if market_price >= price_at_inf - 1e-14:
        return 5.0
    def obj(sigma):
        return black_swaption_price(S0, K, sigma, T, annuity) - market_price
    try:
        vol = brentq(obj, 1e-8, 5.0)
        return float(vol)
    except Exception:
        return float(np.nan)

In [41]:
# -------------------------------
# Market <-> model mapping helpers
# -------------------------------
curve_df = read_curve(curve_csv)
P0 = interp_discount(curve_df)
def payoff_df_fn(t: float) -> float:
    return float(P0(max(1e-8, t)))

caplet_df = pd.read_csv(caplet_csv)
swaption_df = pd.read_csv(swaption_csv)

def extract_atm_caplet_vols(caplet_df: pd.DataFrame) -> Dict[float, float]:
    atm = {}
    for expiry_str, group in caplet_df.groupby("expiry"):
        T = tenor_to_years(expiry_str)
        group = group.copy()
        group["strike_abs"] = np.abs(group["strike"])
        row = group.loc[group["strike_abs"].idxmin()]
        atm[T] = float(row["vol"])
    return atm

def extract_atm_swaption_vols(swap_df: pd.DataFrame) -> Dict[Tuple[float,float], float]:
    atm = {}
    for (expiry_str, tenor_str), group in swap_df.groupby(["expiry", "tenor"]):
        Texp = tenor_to_years(expiry_str)
        Tten = tenor_to_years(tenor_str)
        row = group[group["moneyness"] == "ATM"].iloc[0]
        atm[(Texp, Tten)] = float(row["vol"])
    return atm

atm_caplet = extract_atm_caplet_vols(caplet_df)
atm_swaption = extract_atm_swaption_vols(swaption_df)

def model_caplet_implied_vol(hjm: HJMModel, expiry: float, accrual: float, strike: float, n_paths: int) -> float:
    price = hjm.price_caplet_mc(strike=strike, expiry=expiry, accrual=accrual, payoff_df_fn=payoff_df_fn, n_paths=n_paths)
    target_T = expiry + accrual
    idx = np.argmin(np.abs(hjm.Tgrid - target_T))
    F0 = hjm.f0[idx]
    DF = payoff_df_fn(target_T)
    vol = implied_black_vol_from_price_caplet(price, F0, strike, accrual, DF)
    return vol

def model_swaption_implied_vol(hjm: HJMModel, expiry: float, tenor: float, n_paths: int, payment_freq: float = 1.0) -> float:
    num_payments = int(round(tenor * payment_freq))
    pay_times = np.array([expiry + (k+1)/payment_freq for k in range(num_payments)])
    df_payments = np.array([payoff_df_fn(pt) for pt in pay_times])
    accrual = 1.0 / payment_freq
    annuity = np.sum(df_payments) * accrual
    P_start = payoff_df_fn(expiry)
    P_end = payoff_df_fn(expiry + tenor)
    S0 = (P_start - P_end) / annuity if annuity > 0 else 0.0
    price = hjm.price_swaption_mc(expiry=expiry, swap_tenor=tenor, fixed_rate=S0, payment_freq=payment_freq, payoff_df_fn=payoff_df_fn, n_paths=n_paths)
    vol = implied_black_vol_from_price_swaption(S0=S0, K=S0, T=expiry, annuity=annuity, market_price=price)
    return vol

In [43]:
# ---------------------------------
# Calibration: Stage A & Stage B
# ---------------------------------
def calibrate_stageA(hjm_template: HJMModel, atm_caplet: Dict[float,float], atm_swaption: Dict[Tuple[float,float],float],
                     weights: Dict[str,float], bounds_phi: Dict[str,Tuple[float,float]] = None,
                     fix_beta: Optional[float] = None, n_paths: int = 3): # 3 not 2000 n_paths to save speed on personal laptop
    a0 = hjm_template.params.a
    b0 = hjm_template.params.b
    beta0 = hjm_template.params.beta if fix_beta is None else fix_beta
    if bounds_phi is None:
        bounds_phi = {"a": (1e-5, 0.5), "b": (0.0, 5.0), "beta": (-0.5, 1.5)}
    def objective_x(x):
        a, b, beta = x[0], x[1], (x[2] if fix_beta is None else fix_beta)
        params = HJMParams(a=a, b=b, beta=beta, delta=hjm_template.params.delta,
                           kappa=hjm_template.params.kappa, theta=hjm_template.params.theta,
                           xi=hjm_template.params.xi, rho=hjm_template.params.rho, v0=hjm_template.params.v0)
        hjm = HJMModel(times=hjm_template.Tgrid, f0=hjm_template.f0, params=params, n_steps_per_year=hjm_template.n_steps_per_year, 
                       seed=hjm_template.seed)
        err_sq = 0.0
        for Texp, mkt_vol in atm_caplet.items():
            accrual = 0.25
            try:
                model_vol = model_caplet_implied_vol(hjm, expiry=Texp, accrual=accrual, strike=0.0, n_paths=n_paths)
                if np.isnan(model_vol): continue
                err_sq += weights.get("caplet",1.0) * (model_vol - mkt_vol) ** 2
            except Exception:
                err_sq += 1e-4
        for (Texp, Tten), mkt_vol in atm_swaption.items():
            try:
                model_vol = model_swaption_implied_vol(hjm, expiry=Texp, tenor=Tten, n_paths=n_paths)
                if np.isnan(model_vol): continue
                err_sq += weights.get("swaption",1.0) * (model_vol - mkt_vol) ** 2
            except Exception:
                err_sq += 1e-4
        return float(err_sq)
    x0 = [a0, b0, beta0]
    bnds = [bounds_phi["a"], bounds_phi["b"], bounds_phi["beta"]]
    res = optimize.minimize(objective_x, x0, method="L-BFGS-B", bounds=bnds, options={"maxiter": 30})
    a_fit, b_fit, beta_fit = float(res.x[0]), float(res.x[1]), float(res.x[2]) if fix_beta is None else fix_beta
    result = {"a": a_fit, "b": b_fit, "beta": beta_fit, "fun": float(res.fun), "success": res.success}
    print("Stage A finished:", result)
    return result

def calibrate_stageB(hjm_template: HJMModel, caplet_df: pd.DataFrame, swaption_df: pd.DataFrame, weights: Dict[str,float],
                     bounds_sv: Dict[str,Tuple[float,float]] = None, n_paths: int = 3): # 3 not 2000 n_paths to save speed on personal laptop
    if bounds_sv is None:
        bounds_sv = {"kappa": (1e-4, 10.0), "theta": (1e-6, 2.0), "xi": (1e-4, 3.0), "rho": (-0.99, 0.99), "v0": (1e-6, 2.0)}
    cap_targets = []
    for _, row in caplet_df.iterrows():
        T = tenor_to_years(row["expiry"])
        cap_targets.append(("caplet", T, float(row["strike"]), float(row["vol"]), row["vol_type"]))
    swap_targets = []
    for _, row in swaption_df.iterrows():
        Texp = tenor_to_years(row["expiry"]); Tten = tenor_to_years(row["tenor"])
        swap_targets.append(("swaption", Texp, Tten, row["moneyness"], float(row["vol"]), row["vol_type"]))
    def objective_y(y):
        kappa, theta, xi, rho, v0 = y
        params = HJMParams(a=hjm_template.params.a, b=hjm_template.params.b, beta=hjm_template.params.beta, delta=hjm_template.params.delta,
                           kappa=kappa, theta=theta, xi=xi, rho=rho, v0=v0)
        hjm = HJMModel(times=hjm_template.Tgrid, f0=hjm_template.f0, params=params, n_steps_per_year=hjm_template.n_steps_per_year, 
                       seed=hjm_template.seed)
        err_sq = 0.0
        for ttype, T, strike, mkt_vol, vtype in cap_targets:
            try:
                accrual = 0.25
                price = hjm.price_caplet_mc(strike=strike, expiry=T, accrual=accrual, payoff_df_fn=payoff_df_fn, n_paths=n_paths)
                idx = np.argmin(np.abs(hjm.Tgrid - (T + accrual)))
                F0 = hjm.f0[idx]
                DF = payoff_df_fn(T + accrual)
                model_vol = implied_black_vol_from_price_caplet(price, F0, strike, accrual, DF)
                if np.isnan(model_vol): continue
                err_sq += weights.get("caplet",1.0) * (model_vol - mkt_vol) ** 2
            except Exception:
                err_sq += 1.0
        for ttype, Texp, Tten, mny, mkt_vol, vtype in swap_targets:
            try:
                model_vol = model_swaption_implied_vol(hjm, expiry=Texp, tenor=Tten, n_paths=n_paths)
                if np.isnan(model_vol): continue
                err_sq += weights.get("swaption",1.0) * (model_vol - mkt_vol) ** 2
            except Exception:
                err_sq += 1.0
        reg = 1e-6 * (kappa**2 + theta**2 + xi**2 + rho**2 + v0**2)
        return float(err_sq + reg)
    x0 = [hjm_template.params.kappa, hjm_template.params.theta, hjm_template.params.xi, hjm_template.params.rho, hjm_template.params.v0]
    bnds = [bounds_sv["kappa"], bounds_sv["theta"], bounds_sv["xi"], bounds_sv["rho"], bounds_sv["v0"]]
    res = optimize.minimize(objective_y, x0, method="L-BFGS-B", bounds=bnds, options={"maxiter": 30})
    fitted = {"kappa": float(res.x[0]), "theta": float(res.x[1]), "xi": float(res.x[2]), "rho": float(res.x[3]), "v0": float(res.x[4]), 
              "fun": float(res.fun), "success": res.success}
    print("Stage B finished:", fitted)
    return fitted

In [45]:
# ------------------------------
# Orchestration: main pipeline
# ------------------------------
def run_full_calibration(config: Dict):
    """
    Orchestrate data ingestion, model initialisation, stage A and stage B calibration,
    and produce outputs (plots, csvs, json).
    """
    # 1. read data
    curve_df_local = read_curve(curve_csv)
    cap_df_local = pd.read_csv(caplet_csv)
    swap_df_local = pd.read_csv(swaption_csv)

    # 2. create HJM initial model template with default params
    Tgrid = f0_times
    f0 = f0_vals
    params0 = HJMParams(a=config["phi_init"]["a"], b=config["phi_init"]["b"], beta=config["beta_init"], delta=config["vol_shift"],
                        kappa=1.0, theta=0.04, xi=0.3, rho=-0.5, v0=0.04)
    hjm_template = HJMModel(times=Tgrid, f0=f0, params=params0, n_steps_per_year=config["n_steps_per_year"], seed=config["seed"])

    # 3. stage A: fit phi (a,b) & beta
    weights_stageA = {"caplet": config["cap_weight"], "swaption": config["swaption_weight"]}
    stageA_res = calibrate_stageA(hjm_template, atm_caplet, atm_swaption, weights={"caplet": weights_stageA["caplet"], "swaption": 
                                                                                   weights_stageA["swaption"]},
                                  fix_beta=None if config["beta_init"] is None else config["beta_init"],
                                  n_paths=config["n_paths_stageA"])

    # update template params
    params_afterA = HJMParams(a=stageA_res["a"], b=stageA_res["b"], beta=stageA_res["beta"], delta=params0.delta,
                              kappa=params0.kappa, theta=params0.theta, xi=params0.xi, rho=params0.rho, v0=params0.v0)
    hjm_afterA = HJMModel(times=Tgrid, f0=f0, params=params_afterA, n_steps_per_year=config["n_steps_per_year"], seed=config["seed"])

    # 4. stage B: calibrate stochastic vol
    weights_stageB = {"caplet": config["cap_weight"], "swaption": config["swaption_weight"]}
    stageB_res = calibrate_stageB(hjm_afterA, cap_df_local, swap_df_local, weights=weights_stageB, n_paths=config["n_paths_stageB"])

    # 5. final model with fitted params
    final_params = HJMParams(a=stageA_res["a"], b=stageA_res["b"], beta=stageA_res["beta"], delta=params0.delta,
                             kappa=stageB_res["kappa"], theta=stageB_res["theta"], xi=stageB_res["xi"], rho=stageB_res["rho"], v0=stageB_res["v0"])
    hjm_final = HJMModel(times=Tgrid, f0=f0, params=final_params, n_steps_per_year=config["n_steps_per_year"], seed=config["seed"])

    # 6. diagnostics: compute model vs market ATM vols for both caplets and swaptions
    # Caplets
    model_cap_rows = []
    for Texp, mkt_vol in atm_caplet.items():
        accrual = 0.25
        strike = 0.0
        model_vol = model_caplet_implied_vol(hjm_final, expiry=Texp, accrual=accrual, strike=strike, n_paths=config["n_paths_stageA"])
        model_cap_rows.append({"expiry": Texp, "market_vol": mkt_vol, "model_vol": model_vol, "error": model_vol - mkt_vol})
    df_caps = pd.DataFrame(model_cap_rows)
    caps_out = os.path.join(config["output_dir"], "model_vs_market_caplets.csv")
    df_caps.to_csv(caps_out, index=False)

    # Swaptions
    model_swap_rows = []
    for (Texp, Tten), mkt_vol in atm_swaption.items():
        model_vol = model_swaption_implied_vol(hjm_final, expiry=Texp, tenor=Tten, n_paths=config["n_paths_stageA"])
        model_swap_rows.append({"expiry": Texp, "tenor": Tten, "market_vol": mkt_vol, "model_vol": model_vol, "error": model_vol - mkt_vol})
    df_swaps = pd.DataFrame(model_swap_rows)
    swaps_out = os.path.join(config["output_dir"], "model_vs_market_swaptions.csv")
    df_swaps.to_csv(swaps_out, index=False)

    # 7. compute RMSE
    cap_rmse = float(np.sqrt(np.nanmean(df_caps["error"].dropna() ** 2)))
    swap_rmse = float(np.sqrt(np.nanmean(df_swaps["error"].dropna() ** 2)))
    summary = {"stageA": stageA_res, "stageB": stageB_res, "cap_rmse": cap_rmse, "swap_rmse": swap_rmse}

    # persist calibration_results.json
    out_json = os.path.join(config["output_dir"], "calibration_results.json")
    with open(out_json, "w") as f:
        json.dump(summary, f, indent=2)
    print("Calibration summary saved:", out_json)

    # 8. plots
    # ATM caplet vols
    plt.figure(figsize=(6,4))
    plt.plot(df_caps["expiry"], df_caps["market_vol"], marker='o', label="Market ATM Caplet")
    plt.plot(df_caps["expiry"], df_caps["model_vol"], marker='x', label="Model ATM Caplet")
    plt.xlabel("Expiry (years)"); plt.ylabel("Vol")
    plt.title("ATM Caplet: Market vs Model")
    plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(config["output_dir"], "plots", "atm_caplet_fit.png"))
    plt.close()

    # ATM swaption vols (pick by tenor)
    plt.figure(figsize=(8,5))
    for tenor in sorted(set(df_swaps["tenor"])):
        sel = df_swaps[df_swaps["tenor"]==tenor]
        plt.plot(sel["expiry"], sel["market_vol"], marker='o', label=f"Market tenor {tenor}y")
        plt.plot(sel["expiry"], sel["model_vol"], marker='x', linestyle='--', label=f"Model tenor {tenor}y")
    plt.xlabel("Expiry (years)"); plt.ylabel("Vol"); plt.title("ATM Swaption Surface: Market vs Model")
    plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(config["output_dir"], "plots", "atm_swaption_fit.png"))
    plt.close()

    # 9. save parameters JSON
    params_out = os.path.join(config["output_dir"], "calibrated_parameters.json")
    with open(params_out, "w") as f:
        json.dump({
            "final_params": final_params.__dict__,
            "rmse": {"cap": cap_rmse, "swap": swap_rmse}
        }, f, indent=2)
    print("Saved calibrated parameters:", params_out)

    # 10. print summary
    print("\n=== Calibration SUMMARY ===")
    print(f"Caplet RMSE: {cap_rmse:.6f}")
    print(f"Swaption RMSE: {swap_rmse:.6f}")
    print("Final parameters:")
    print(json.dumps(final_params.__dict__, indent=2))
    return summary

In [47]:
# -----------------------
# Run the pipeline (demo)
# -----------------------
if __name__ == "__main__":
    print("Running HJM calibration demo (fast mode). This will create outputs in", CONFIG["output_dir"])
    # remove old outputs for a fresh run (optional)
    if os.path.exists(CONFIG["output_dir"]):
        try:
            shutil.rmtree(CONFIG["output_dir"])
        except Exception:
            pass
    ensure_dirs()
    run_summary = run_full_calibration(CONFIG)
    print("\nDone. Key outputs in", CONFIG["output_dir"])

Running HJM calibration demo (fast mode). This will create outputs in outputs
Stage A finished: {'a': 0.02, 'b': 0.5, 'beta': 0.0, 'fun': 22.343583090210004, 'success': True}
Stage B finished: {'kappa': 1.0007740027924705, 'theta': 0.006796422922430871, 'xi': 0.05104995710985319, 'rho': -0.6371888607385758, 'v0': 0.006796422922430871, 'fun': 15.237724876679492, 'success': False}
Calibration summary saved: outputs\calibration_results.json
Saved calibrated parameters: outputs\calibrated_parameters.json

=== Calibration SUMMARY ===
Caplet RMSE: 2.877161
Swaption RMSE: 0.014803
Final parameters:
{
  "a": 0.02,
  "b": 0.5,
  "beta": 0.0,
  "delta": 0.0001,
  "kappa": 1.0007740027924705,
  "theta": 0.006796422922430871,
  "xi": 0.05104995710985319,
  "rho": -0.6371888607385758,
  "v0": 0.006796422922430871
}

Done. Key outputs in outputs
